In [3]:
%reload_ext autoreload
%matplotlib inline

In [12]:
%autoreload 2

import sys,os,glob
import psutil

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
from pprint import pprint
import datetime as dt


from matplotlib.colors import LogNorm
from matplotlib.gridspec import GridSpec
import cartopy.crs as ccrs

import datetime as dt
import re
import gc
import matplotlib.animation as animation
import warnings
import pickle
from scipy.interpolate import interp1d


#-- Own settings

from settings import *

## current script object
thismodule = sys.modules[__name__]
workdir = os.getcwd()
# Add own module library to path
moduledir, fcndir = defineDir(workdir)

# load own libraries
from conditionalstats import *
from conditionalstats_chunked import *
from plot1D import *

# own DYAMOND functions
from fcns_load_DYAMOND_SAM import *
# own TOOCAN functions
from load_TOOCAN_DYAMOND_modif_BF import load_TOOCAN_DYAMOND

Own modules available: ['conditionalstats', 'plot2D', 'plotCondPDFs', 'plot1D', 'conditionalstats_chunked']
Own modules available: ['fcns_load_DYAMOND_SAM', 'load_TOOCAN_DYAMOND_modif', 'load_TOOCAN_DYAMOND', 'load_TOOCAN_DYAMOND_modif_BF', 'casestudy']


# Pacific ITCZ

1. Distribution de pluie (au cours du temps)
2. Distribution de pluie conditionnée (à TOOCAN, en fonction du temps)
3. Pente des distributions
4. Fréquence d'occurrence des systèmes (en fonction du temps)
5. 

In [5]:
path_dy = '/data/bfildier/DYAMOND_REGIONS/205E_250E_0N_20N/SAM/'
path_dy_2D = os.path.join(path_dy,'2D')

In [24]:
regions = [regionNameFromCoord(box) for box in [box_1,box_2,box_3]]+['tropics']
masks = ['all','TOOCAN','PW_40']
varids = 'pr','PW'
varids_DY = 'Prec','PW'
i_t_0 = 832
i_t_end = 1917

In [30]:
# Load precomputed distributions

dict_distribs = {}

for region in regions:
    
    # init
    dict_distribs[region] = {}

    for varid,varid_DY in zip(varids,varids_DY):

        # init
        dict_distribs[region][varid] = {}
        
        for mask in masks:
            
            print(region,varid,mask)
            
            # init
            dict_distribs[region][varid][mask] = {}
            
            for i_t in np.arange(i_t_0,i_t_end+1):
                
                if i_t%100 == 0 or i_t in [i_t_0,i_t_end]:
                    print(i_t,end='..')
                
                filename = '../results/%s/%s/%s/time_slices/dist_%s_t_%d.pickle'%(region,varid_DY,mask,varid,i_t)

                # load if exists
                if len(glob.glob(filename)) > 0:
                    
                    dict_distribs[region][varid][mask][i_t] = pickle.load(open(filename,'rb'))
                
            print()

310E_340E_0N_20N pr all
832..900..1000..1100..1200..1300..1400..1500..1600..1700..1800..1900..1917..
310E_340E_0N_20N pr TOOCAN
832..900..1000..1100..1200..1300..1400..1500..1600..1700..1800..1900..1917..
310E_340E_0N_20N pr PW_40
832..900..1000..1100..1200..1300..1400..1500..1600..1700..1800..1900..1917..
310E_340E_0N_20N PW all
832..900..1000..1100..1200..1300..1400..1500..1600..1700..1800..1900..1917..
310E_340E_0N_20N PW TOOCAN
832..900..1000..1100..1200..1300..1400..1500..1600..1700..1800..1900..1917..
310E_340E_0N_20N PW PW_40
832..900..1000..1100..1200..1300..1400..1500..1600..1700..1800..1900..1917..
205E_250E_0N_20N pr all
832..900..1000..1100..1200..1300..1400..1500..1600..1700..1800..1900..1917..
205E_250E_0N_20N pr TOOCAN
832..900..1000..1100..1200..1300..1400..1500..1600..1700..1800..1900..1917..
205E_250E_0N_20N pr PW_40
832..900..1000..1100..1200..1300..1400..1500..1600..1700..1800..1900..1917..
205E_250E_0N_20N PW all
832..900..1000..1100..1200..1300..1400..1500..1600..

In [ ]:
# Compute full distributions



In [31]:
# Calculate distribution slopes


[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 4.09393311e-02
 4.09545898e-02 1.22817993e-01 2.04711914e-01 4.09393311e-01
 6.95975304e-01 1.10536194e+00 1.63760376e+00 2.37451172e+00
 3.31613159e+00 4.42150879e+00 5.81344604e+00 7.20541382e+00
 8.76107788e+00 1.02741420e+01 1.19544373e+01 1.36329346e+01
 1.52705383e+01 1.67853014e+01 1.83410187e+01 1.99376831e+01
 2.16571193e+01 2.33294712e+01 2.51779441e+01 2.62377836e+01
 2.75568396e+01 2.92257544e+01 3.03363777e+01 3.24550516e+01
 3.38429768e+01 3.52572837e+01 3.61660069e+01 3.71361640e+01
 3.81007354e+01 3.90368318e+01 3.94768882e+01 4.14782397e+01
 4.25433793e+01 4.39013030e+01 4.45229441e+01 4.54758926e+01]
[ 0.          0.          0.          0.          0.          0.08187866
  0.28657532  0.6550293   1.18722534  1.84225464  2.74298096  3.84835815
  5.19934082  6.83694458  8.39267136  9.97010081 11.66784668